In [6]:
from pathlib import Path
from enum import Enum
import cv2
from cv2 import imwrite
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import preprocessing
from time import time
from scipy.optimize import curve_fit
%matplotlib inline
import scipy.stats
from util import parse_fn, is_bg
from scipy import stats
from scipy.stats import skewnorm
from scipy.special import erf
from scipy.special import voigt_profile
import re

# Input Here:

## information of picture

In [7]:
# HEADER_LEN = 152
# X_DIM = 480
# Y_DIM = 640
# X_left_boundry = 185
# X_right_boundry = 235
# Y_left_boundry = 355
# Y_right_boundry = 505

# frame_rate = 92.59
# shutter_delay = 0.01 # second
# interestd_position = 4 # mm
# pixel_size = 0.00108 
# beamwidth = 88200


# exposure time = 5ms

## melting power

### Si

In [8]:
# Si_melting_W = {
#     '103': 41.5,
#     '13': 35.4,
#     '155': 43.7,
#     '190': 45.5,
#     '20': 35.5,
#     '234': 47,
#     '30': 36.3,
#     '352': 52.2,
#     '45': 37.1,
#     '68': 39.2,
#     '9': 35.3 
    
# }

### gold

In [9]:
# gold_melting_W = {
#     '103': 37,
#     '13': 30.2,
#     '155': 38.9,
#     '190': 40.3,
#     '20': 30.8,
#     '234': 41.9,
#     '30': 31.9,
#     '352': 46.2,
#     '45': 33.5,
#     '68': 35.2,
#     '9': 29.9
    
# }

In [10]:
# 0124 diode 

# HEADER_LEN = 0
# X_DIM = 1200
# Y_DIM = 1920
# X_left_boundry = 465
# X_right_boundry = 555
# Y_left_boundry = 520
# Y_right_boundry = 960

## picture to blue light signal intensity

In [11]:
def read_raw_to_intensity(file):
    
    with open(file, "rb") as rawimage:
    
        bayer_image = np.fromfile(rawimage, np.dtype('u1'), offset = HEADER_LEN)
        fst_byte, scd_byte = bayer_image.reshape((bayer_image.shape[0]//2, 2)).astype(np.uint16).T
        raw_pixel = (fst_byte + (scd_byte << 8)).reshape((X_DIM, Y_DIM))

    return raw_pixel


def intensity_to_blue_16(intensity):
    intensity_uint16 = np.array(intensity, dtype = np.uint16)
    color = cv2.cvtColor(intensity_uint16, cv2.COLOR_BayerGRBG2RGB)
    blue = color[:,:,2]
    return blue


def get_blue(file):
    raw_pixel = read_raw_to_intensity(file)
    blue = intensity_to_blue_16(raw_pixel)
    return blue.astype(np.float64)

## get the bright region

In [12]:
def get_bright(pure_blue):
    data = pure_blue[X_left_boundry:X_right_boundry, Y_left_boundry:Y_right_boundry]
    return data

## get blue signal of interested region : get_data(file)

In [13]:
# 拿到我要的範圍的blue
def get_data(file):
    blue = get_blue(file)
    bright = get_bright(blue)
    return bright

## get final fitting peak dR / R (one value) of a frame

In [14]:
def peak_dRoverR(file, b_file, func, maxs_func):
    dRoverR = get_dRoverR(file, b_file)
    maxs = get_maxs(dRoverR, func)
    peak = get_max(maxs, maxs_func)
    return peak

In [15]:
def get_peak_dRoverR(file, back_intensity, func, maxs_func):
    dRoverR = get_dR_over_R(file, back_intensity)
    maxs = get_maxs(dRoverR, func)
    peak = get_max(maxs, maxs_func)
    return peak

## get dR / R of every point of interested in a frame

In [16]:
def get_dRoverR(file, b_file):
    blue = get_data(file)
    b_blue = get_data(b_file)
    dRoverR = (blue - b_blue) / b_blue
    return dRoverR

### 得到一張圖的需要的部分的dR/R

In [17]:
def get_dR_over_R(file, back_intensity):
    blue = get_data(file)
    b_blue = get_bright(back_intensity)
    dRoverR = (blue - b_blue) / b_blue
    return dRoverR

# get fitting line for lateral profile

In [18]:
def get_laterial_fit_line(file, back_intensity, func):
    dRoverR = get_dR_over_R(file, back_intensity)
    maxs = get_maxs(dRoverR, func)
    
    return maxs

In [1]:
def generate_consecutive_integers(center, length):
    """Generates a list of consecutive integers centered around a given value."""
    start = center - length // 2
    return [i for i in range(start, start + length)]
